# The Error Correction Module

The error correction module will take input from OCR and will try to match each word with the word present in the dictionary.   

I have tried to to do that with three methods :  
1. **Based on transformations:** The least number of transformations will be given the most preference.    
2. **Based on the probability of the word:** Here, I have allows a 'Fuzzy window'/number of transformations as 2. So, for up to 2 transformations, it will try to calculate the combination of the word and its probability. And the top 3 most probable words will be presented.  
3. **Using the N-gram Language model :** In the N-gram model I have calculated N-grams for the whole corpus, And for the test word, It will generate up to N-1 grams will generate their probabilities. From these probabilities, we can generate the approach of which character will be present next after the mistype.  


	   

### Sorting Imports : 

In [15]:
import nltk
from nltk import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
import re

For Imports, I have used NLTK for Tokenizing and Collections for Counter

### File Handling :

In [16]:
def FileLoad(filename):
    file = open(filename, "r")
    data = file.read()
    file.close()

    return data

In FileLoad(), I am taking a file from Corpus and read the file

In [17]:
textInput = FileLoad("europarl-v7.bg-en.en")

Here, I have taken a file from Europarl corpus 'europarl-v7.bg-en.en'

### Tokenize :

In [18]:
words = word_tokenize(textInput)

Here, I am using NLTK's built-in function (word_tokenize()) which takes text as an argument and returns a list of tokenized words.

### Data Preprocessing : 

In [19]:
words = [word.lower() for word in words if word.isalpha()]

Here, I am **preprocessing** the list of words by converting all the words into lowercase letters and only keeping the tokens which have alphabets in them like words, I have used word.isalpha() for this in order to remove the noise (. , / , "" etc..).  
Here I have removed everything except words e.g. : Numbers and Punctuation.  
But if we want to remove them separately, We can also use **isdigit()** for numbers or **re.sub()** for Regular expressions.    


In [20]:
StopW = set(stopwords.words("english"))

words = [w for w in words if not w in StopW]

Here, I have used NLTK's **Stopwords** functionality to remove the stopwords such as the, is, at, which, on, etc. in order to further clean the list of tokens.  

In [21]:
lemmatizer = WordNetLemmatizer()

words = [lemmatizer.lemmatize(w) for w in words]

Here, I have used NLTK's Lemmatizers to only keep the root of the words in order to decrease the frequency of similar words.    

**Stemming vs Lemmatization :**  
In stemming, We remove the prefixes and suffixes to keep the stem of the word, even if the word is not valid.    
This is done in order to keep a group of the same words with different grammar infliction together.      
Generally, Stemmers have an algorithm so they are fast as compared to Lemmatizers.  
In Lemmatizing, the algorithm makes sure that after the reduction is done, the word (root) still has a meaning/the word is present in the language.  

Stemming vs Lemmatizing depends on the type of application.  
As we are working with an OCR language application, language is important and so used lemmatization as it uses a corpus to match root forms.  
  

In [22]:
WordDict = Counter(words)

Counter to create a dictionary of words with frequency.

In [23]:
TotalWords = sum(WordDict.values())


def probability(word):
    prob = (WordDict[word] / TotalWords) * 100
    return prob


def edits1(word):
    letters = "abcdefghijklmnopqrstuvwxyz"
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)


def edits2(word):
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

Here, I have defined 3 functions :   
1. probability: Which calculates the probability of the word.  
2. edits1: all the combination of words that are possible with 1 transformation  
e.g.: splitting, deleting, transposing, replacing and inserting  
3. edits2: all the combination of words that are possible with 2 transformations  

Here the number of transformations represents the Edit distance between the two words.  
Edit distance compares the two words and tries to quantify how dissimilar they are and how many operations it will take to match them.  

Here I have kept maximum **Edit Distance of 2**.  

In [24]:
def suggestion1(word):
    if word in WordDict:
        #Checks if the word is present in the dictionary
        print("Given word '{}' is Correct..!!!!".format(word))
    else:
        #Edit distance = 1
        firstpref = edits1(word)

        for i in firstpref:
            if i in WordDict:
                prob = probability(i)
                print(
                    "'{}' is the closest word in the corpus".format(
                        i
                    )
                )
                return
        
        #Edit Distance = 2
        secondpref = edits2(word)

        for j in secondpref:
            if j in WordDict:
                prob = probability(j)
                print(
                    "'{}' is the closest word in the corpus".format(
                        j
                    )
                )
                return
            else:
                print("No word is closest to the given word - {}".format(word))
                return


suggestion1("laywer")


'''
Output : 

'layer' is the closest word in the corpus

'''

'layer' is the closest word in the corpus


"\nOutput : \n\n'layer' is the closest word in the corpus\n\n"

In the code block above, I have defined a function Suggestion1() which suggests the closest alternative to the given word.      
The Function check if the given word is present in the dictionary. If not, Then it takes the Edit distance as 1 and tries to calculate the combination of words closest to the given word with 1 transformation.  
And displays the word.    
If there are no words in the dictionary with 1 transformation, Then we use Edits2 to make Edit Distance = 2 and make those words to check the closest one.    
  
In this function, We rely on the Error model on the terms of Edit Distance (If words in Edit distance = 1 dont match, Then go to Edit Distance = 2 or else no results).  
  
Here, Probability is not taken into account as each word at same edit distance will have the same p(w|c).  
  
  

In [25]:
def suggestion2(word):
    prob = {}
    if word in WordDict:
        print("Given word '{}' is Correct..!!!!".format(word))
    else:
        firstpref = edits1(word)

        for i in firstpref:
            if i in WordDict:
                prob[i] = probability(i)

        secondpref = edits2(word)

        for j in secondpref:
            if j in WordDict:
                prob[j] = probability(j)
        # print(prob)

    sortedSuggest = sorted(prob.items(), key=lambda x: x[1], reverse=True)
    #Sorting in descending order of the probability

    print(
        "Top 3 suggestions for the misinterprited word are :\n",
        "1. '{}' with probability : {} %".format(
            sortedSuggest[0][0], sortedSuggest[0][1]
        ),
        "\n",
        "2. '{}' with probability : {} %".format(
            sortedSuggest[1][0], sortedSuggest[1][1]
        ),
        "\n",
        "3. '{}' with probability : {} %".format(
            sortedSuggest[2][0], sortedSuggest[2][1]
        ),
        "\n",
    )


suggestion2("laywer")



'''
Output : 

Top 3 suggestions for the misinterprited word are :
 1. 'later' with probability : 0.017776651158326995 % 
 2. 'lower' with probability : 0.013484339033537334 % 
 3. 'player' with probability : 0.013423598767620497 % 

'''

Top 3 suggestions for the misinterprited word are :
 1. 'later' with probability : 0.017776651158326995 % 
 2. 'lower' with probability : 0.013484339033537334 % 
 3. 'player' with probability : 0.013423598767620497 % 



"\nOutput : \n\nTop 3 suggestions for the misinterprited word are :\n 1. 'later' with probability : 0.017776651158326995 % \n 2. 'lower' with probability : 0.013484339033537334 % \n 3. 'player' with probability : 0.013423598767620497 % \n\n"

In Suggestion2(), We solely rely on the probability of the combination of words in a dictionary.  
  
In the function, We have allowed the fuzzy window of up to 2 transformations and calculated the probability of each word and displayed the top 3 suggestions with the highest probability.  

In [26]:
contents = "Science Fantasy was a cacti playing British 3 fantasy and science fiction magazine, launched in 1950 by Nova Publications. 4 John Carnell edited the magazine beginning with the third issue, typically running a long lead novelette along with several shorter stories. Prominent contributors in the 1950s included John Brunner, Ken Bulmer, and Brian Aldiss, whose first novel Nonstop appeared (in an early version) in the February 1956 issue. Fantasy stories began to appear more frequently during the latter half of the 1950s, and in the early 1960s Carnell began to publish Thomas Burnett Swann's well-received historical fantasies. In the early 1960s Carnell's efforts were rewarded with three consecutive Hugo nominations for best magazine. After Nova went out of business in early 1964, Roberts & Vinter took over as publishers until 1967. Kyril Bonfiglioli, the editor, attracted new writers, including Keith Roberts, Brian Stableford and Josephine Saxton. In the opinion of science fiction historian Mike Ashley, the final year of the magazine, when it was renamed Impulse, included some of the best material ever published in a British science fiction magazine."

words = word_tokenize(contents)

words = [word.lower() for word in words if word.isalpha()]

StopW = set(stopwords.words("english"))

words = [w for w in words if not w in StopW]

# lemmatizing
lemmatizer = WordNetLemmatizer()

words = [lemmatizer.lemmatize(w) for w in words]

s = ""
s = s.join(words)

ngrams = [s[i : i + n] for n in range(1, 8) for i in range(len(s) - n + 1)]
#ngrams from unigrams up to 7-grams

# print(ngrams)

dictall = Counter(ngrams)


frequency = dictall.__sizeof__()


TestWord = "fictional"



if TestWord in words:
    print("Found it!!,word is correct")

elif TestWord in dictall:

    probability = dictall[TestWord] / frequency

    print("Probability of {} is : {} ".format(TestWord, probability))

else:

    for n in range(1, len(TestWord)):

        if TestWord[:-n] in dictall:

            smoothing = 1
            probability = (dictall[TestWord[:-n]] + smoothing) / frequency

            print("Probability of {} is : {} ".format(TestWord[:-n], probability))

        else:
            print("Nothing is found  at {} ".format(TestWord[:-n]))

'''
Output : 

Nothing is found  at fictiona 
Probability of fiction is : 2.7109086965950985e-05 
Probability of fictio is : 2.7109086965950985e-05 
Probability of ficti is : 2.7109086965950985e-05 
Probability of fict is : 2.7109086965950985e-05 
Probability of fic is : 2.7109086965950985e-05 
Probability of fi is : 4.744090219041423e-05 
Probability of f is : 0.0001152136196052917 

'''

Nothing is found  at fictiona 
Probability of fiction is : 2.7109086965950985e-05 
Probability of fictio is : 2.7109086965950985e-05 
Probability of ficti is : 2.7109086965950985e-05 
Probability of fict is : 2.7109086965950985e-05 
Probability of fic is : 2.7109086965950985e-05 
Probability of fi is : 4.744090219041423e-05 
Probability of f is : 0.0001152136196052917 


'\nOutput : \n\nNothing is found  at fictiona \nProbability of fiction is : 2.7109086965950985e-05 \nProbability of fictio is : 2.7109086965950985e-05 \nProbability of ficti is : 2.7109086965950985e-05 \nProbability of fict is : 2.7109086965950985e-05 \nProbability of fic is : 2.7109086965950985e-05 \nProbability of fi is : 4.744090219041423e-05 \nProbability of f is : 0.0001152136196052917 \n\n'

In this Code block, I have used an N-gram language model which calculates the N-gram of the corpus and then compares it with the word given.  
It tries to produce the probability of N-grams for the given word starting from the last word and reducting 1 letter.  
  
This way, We can use the N-grams to predict the next letter present or should be present in the word using the probability of that N-gram.  
  
(Note : I have used a Test string 'Contents' to show the working of N-gram and not corpus due to limited configuration of my system)  
 